# Pegando os links dos anos

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import time

chrome_options = Options()
#chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

    
url = "http://www4.planalto.gov.br/legislacao/portal-legis/legislacao-1/decretos-leis"

driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver.get(url)

driver.implicitly_wait(5)

time.sleep(2)

soup = BeautifulSoup(driver.page_source, 'lxml')

href = ' '.join([str(elem) for elem in soup]).split('<span class="internal-link">')

links_ano = []
for i in range(0,len(href)):
    if 'class="internal-link"' in str(href[i]):
        result = str(href[i]).split('href="')[1].split('" target="')[0]
        links_ano.append(result)
        
links_ano

<ipython-input-1-16d9c5caf3b3>:19: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


['http://www4.planalto.gov.br/legislacao/portal-legis/legislacao-1/decretos-leis/1937-a-1946-decretos-leis-1',
 'http://www4.planalto.gov.br/legislacao/portal-legis/legislacao-1/decretos-leis/1965-a-1988-decretos-leis']

# Pegando os links dos PDF's

In [2]:
chrome_options = Options()
#chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

link_pdf = []

for url in links_ano:
    
    driver.get(url)

    driver.implicitly_wait(5)

    time.sleep(5)

    soup = BeautifulSoup(driver.page_source, 'lxml')

    href = ' '.join([str(elem) for elem in soup]).split('<tr class="visaoQuadrosTr">')

    link_pdf_ano = []

    for i in range(0,len(href)):
        if '<td class="visaoQuadrosTd">\n<a' in str(href[i]):
            result = str(href[i]).split('href="')[1].split('">\n')[0]
            link_pdf_ano.append(result)
        
    link_pdf.append(link_pdf_ano)
    
# Transformando numa única lista 

flat_list_pdf = [item for sublist in link_pdf for item in sublist]

<ipython-input-2-508c81c971be>:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


# Pegando o conteudo textual e o link de redirecionamento 

In [3]:
import re 

chrome_options = Options()
#chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Conteudo_textual = []
Conteudo_link = []

for url in flat_list_pdf:
    
    try:
        
        driver.get(url)

        driver.implicitly_wait(5)

        time.sleep(3)

        soup = BeautifulSoup(driver.page_source, 'lxml')
        
        # Pegando o direcionamento de link
        
        Link = str(soup.find_all('a')).split('href="')[1].split('">')[0]
        
        if len(Link) != 0:
    
            Conteudo = soup.find_all('p')
    
            texto = re.sub("[\<\[].*?[\>\]]", "", str(Conteudo)).replace('\n','').replace(', \xa0','')
    
            Conteudo_textual.append(texto)
    
            Conteudo_link.append(Link)
        
    except:
        
        print(f'Erro de leitura no link: {url}')
        
for i in range(0,len(Conteudo_link)):
    if '" style="color:' in str(Conteudo_link[i]):
        Conteudo_link[i] = str(Conteudo_link[i]).split('" style="color:')[0]
        
for i in range(0,len(Conteudo_link)):
    if 'Document&amp' in str(Conteudo_link[i]):
        Conteudo_link[i] = str(Conteudo_link[i]).split('Document&amp')[0]+'Document'

<ipython-input-3-0f10df88337b>:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


# Extraindo informações adicionais

In [4]:
chrome_options = Options()
#chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Data_assinatura = []
Ementa = []
Situacao = []
Chefe_governo = []
Origem = []
Data_publicacao = []
Fonte = []
Referenda = []
Alteracao = []
Veto = []
Assunto = []
Classificacao_direito = []
Observacao = []
Correlacao = []
Titulo = []
error_site = []

for url in Conteudo_link:
    try:
        
        driver.get(url)

        driver.implicitly_wait(5)

        time.sleep(3)

        soup = BeautifulSoup(driver.page_source, 'lxml')
    
        items_titulo = soup.find_all("h1", {"class": "text-center font-weight-bold title text-uppercase"})
        Titulo.append(str(items_titulo[0]).split('text-uppercase">')[1].split('</h1>')[0])
    
        items = soup.find_all("li", {"class": "list-group-item border-0 p-0"})
        href = ' '.join([str(elem) for elem in items]).split('<div class="col-sm-2 label p-2">')

        # Separando as informações textuais
    
        for i in range(0,len(href)):
            try:
                if 'Data de assinatura:' in str(href[i]):
                    result = str(href[i]).split('text-justify">')[1].split('/div>')[0].replace('\t','').replace('\n','').replace('  ','').replace('<','').replace('br/>','')
                    Data_assinatura.append(result)
            except:
                Data_assinatura.append('')
            
    
        for i in range(0,len(href)):
            try:
                if 'Ementa:' in str(href[i]):
                    result = str(href[i]).split('text-justify">')[1].split('/div>')[0].replace('\t','').replace('\n','').replace('  ','').replace('<','').replace('br/>','')
                    Ementa.append(result)
            except:
                Ementa.append('')
        
    
        for i in range(0,len(href)):
            try:
                if 'Situação:' in str(href[i]):
                    result = str(href[i]).split('text-justify">')[1].split('/div>')[0].replace('\t','').replace('\n','').replace('  ','').replace('<','').replace('br/>','')
                    Situacao.append(result)
            except:
                Situacao.append('')
            
    
        for i in range(0,len(href)):
            try:
                if 'Chefe de Governo:' in str(href[i]):
                    result = str(href[i]).split('text-justify">')[1].split('/div>')[0].replace('\t','').replace('\n','').replace('  ','').replace('<','').replace('br/>','')
                    Chefe_governo.append(result)
            except:
            
                Chefe_governo.append('')
        
    
        for i in range(0,len(href)):
            try:
                if 'Origem:' in str(href[i]):
                    result = str(href[i]).split('text-justify">')[1].split('/div>')[0].replace('\t','').replace('\n','').replace('  ','').replace('<','').replace('br/>','')
                    Origem.append(result)
            except:
            
                Origem.append('')
        
    
        for i in range(0,len(href)):
            try:
                if 'Data de Publicação:' in str(href[i]):
                    result = str(href[i]).split('text-justify">')[1].split('/div>')[0].replace('\t','').replace('\n','').replace('  ','').replace('<','').replace('br/>','')
                    Data_publicacao.append(result)
            except:
            
                Data_publicacao.append('')
            
    
        for i in range(0,len(href)):
            try:
                try:
                    if 'Fonte:' in str(href[i]):
                        result = str(href[i]).split('text-justify">')[1].split('/div>')[0].replace('\t','').replace('\n','').replace('  ','').replace('<','').replace('br/>','').split('href="')[1].split('" target=')[0].replace('amp;','')
                        Fonte.append(result)
            
                except:
                    result = str(href[i]).split('text-justify">')[1].split('/div>')[0].replace('\t','').replace('\n','').replace('  ','').replace('<','').replace('br/>','').replace('amp;','')
                    Fonte.append(result)
            except:
                Fonte.append('')
        
    
        for i in range(0,len(href)):
            try:
                if 'Referenda:' in str(href[i]):
                    result = str(href[i]).split('text-justify">')[1].split('/div>')[0].replace('\t','').replace('\n','').replace('  ','').replace('<','').replace('br/>','')
                    Referenda.append(result)
            except:
                Referenda.append('')
        
    
        for i in range(0,len(href)):
            try:
                if 'Alteração:' in str(href[i]):
                    result = str(href[i]).split('text-justify">')[1].split('/div>')[0].replace('\t','').replace('\n','').replace('  ','').replace('<','').replace('br/>','')
                    Alteracao.append(result)
            except:
                Alteracao.append('')
        
    
        for i in range(0,len(href)):
            try:
                if 'Correlação:' in str(href[i]):
                    result = str(href[i]).split('text-justify">')[1].split('/div>')[0].replace('\t','').replace('\n','').replace('  ','').replace('<','').replace('br/>','')
                    Correlacao.append(result)
            except:
                Correlacao.append('')
        
    
        for i in range(0,len(href)):
            try:
                if 'Veto:' in str(href[i]):
                    result = str(href[i]).split('text-justify">')[1].split('/div>')[0].replace('\t','').replace('\n','').replace('  ','').replace('<','').replace('br/>','')
                    Veto.append(result)
            except:
                Veto.append('')
        
    
        for i in range(0,len(href)):
            try:
                if 'Assunto:' in str(href[i]):
                    result = str(href[i]).split('text-justify">')[1].split('/div>')[0].replace('\t','').replace('\n','').replace('  ','').replace('<','').replace('br/>','')
                    Assunto.append(result)
            except:
                Assunto.append('')
    
        for i in range(0,len(href)):
            try:
                if 'Classificação de direito:' in str(href[i]):
                    result = str(href[i]).split('text-justify">')[1].split('/div>')[0].replace('\t','').replace('\n','').replace('  ','').replace('<','').replace('br/>','')
                    Classificacao_direito.append(result)
            except:
                Classificacao_direito.append('')
        
    
        for i in range(0,len(href)):
            try:
                if 'Observação:' in str(href[i]):
                    result = str(href[i]).split('text-justify">')[1].split('/div>')[0].replace('\t','').replace('\n','').replace('  ','').replace('<','').replace('br/>','')
                    Observacao.append(result)
            except:
                Observacao.append('')
                
    except:
        
        error_site.append(url)
        print(f'Erro na obtenção dos dados referentes ao link: {url}')
        
# Realizando a limpeza dos links com problemas

links_indice = []

for i in range(0,len(error_site)):
    
    result = Conteudo_link.index(error_site[i])
    links_indice.append(result)

# Removendo os links

for i in range(0,len(links_indice)):
        
    del Conteudo_link[links_indice[i]]
    
# Removendo o conteudo textual

for i in range(0,len(links_indice)):
        
    del Conteudo_textual[links_indice[i]]

<ipython-input-4-a0b91e134edc>:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


Erro na obtenção dos dados referentes ao link: http://legislacao.planalto.gov.br/legisla/legislacao.nsf/Viw_Identificacao/del%208.620-1946?OpenDocument
Erro na obtenção dos dados referentes ao link: http://legislacao.planalto.gov.br/leisla/legislacao.nsf/Viw_Identificacao/DEL%205.315-1943?OpenDocument
Erro na obtenção dos dados referentes ao link: http://legislacao.planalto.gov.br/legisla/legislacao.nsf/Viw_Identificacao/DEL%205.084A-1942?OpenDocument
Erro na obtenção dos dados referentes ao link: http://legislacao.planalto.gov.br/legisla/legislacao.nsf/Viw_Identificacao/DEL%201.771-1979?OpenDocument


# Exportando em CSV

In [7]:
# Criando um DataFrame para alocar os outputs

import pandas as pd

BANCO = pd.DataFrame (Titulo ,columns=['Decreto-Lei'])
BANCO['Link'] = Conteudo_link
BANCO['Texto'] = Conteudo_textual
BANCO['Data de assinatura'] = Data_assinatura
BANCO['Ementa'] = Ementa
BANCO['Situação'] = Situacao
BANCO['Chefe de Governo'] = Chefe_governo
BANCO['Origem'] = Origem
BANCO['Data de publicação'] = Data_publicacao
BANCO['Fonte'] = Fonte
BANCO['Referenda'] = Referenda
BANCO['Alteração'] = Alteracao
BANCO['Veto'] = Veto
BANCO['Classificação de direito'] = Classificacao_direito
BANCO['Observação'] = Observacao
BANCO['Correlação'] = Correlacao

# Removendo páginas bloqueadas pelo governo 

BANCO = BANCO[BANCO['Texto'] != ']']

excelfilename = 'Decretos_leis'+ time.strftime("%d-%m-%Y") +".csv"

BANCO.to_csv(excelfilename2, index=False, encoding='utf-8-sig', sep = '汉')

In [10]:
BANCO

,Decreto-Lei,Link,Texto,Data de assinatura,Ementa,Situação,Chefe de Governo,Origem,Data de publicação,Fonte,Referenda,Alteração,Veto,Classificação de direito,Observação,Correlação
0,Decreto-Lei nº 9.914 de 17 de setembro de 1946,http://legislacao.planalto.gov.br/legisla/legi...,", Presidência da RepúblicaCasa CivilSubchef...",17 de Setembro de 1946,"ESCLARECE DISPOSITIVOS DO DECRETO-LEI 9.840, D...",Não consta revogação expressa,EURICO GASPAR DUTRA,Executivo,17 de Setembro de 1946,DOFC 17 09 1946 000000 0,---,,---,---,---,
1,Decreto-Lei nº 9.913 de 17 de setembro de 1946,http://legislacao.planalto.gov.br/legisla/legi...,", Presidência da RepúblicaCasa CivilSubchef...",17 de Setembro de 1946,,Não consta revogação expressa,EURICO GASPAR DUTRA,Executivo,17 de Setembro de 1946,DOFC 17 09 1946 000000 0,IMOVEL.,---,---,---,---,---
2,Decreto-Lei nº 9.912 de 17 de setembro de 1946,http://legislacao.planalto.gov.br/legisla/legi...,", Presidência da RepúblicaCasa CivilSubchef...",17 de Setembro de 1946,,Não consta revogação expressa,EURICO GASPAR DUTRA,Executivo,17 de Setembro de 1946,DOFC 17 09 1946 000000 0,ESPORTE.,---,---,---,---,---
3,Decreto-Lei nº 9.911 de 17 de setembro de 1946,http://legislacao.planalto.gov.br/legisla/legi...,", Presidência da RepúblicaCasa CivilSubchef...",17 de Setembro de 1946,,Não consta revogação expressa,EURICO GASPAR DUTRA,Executivo,17 de Setembro de 1946,DOFC 17 09 1946 000000 0,(MM); CREDITO SUPLEMENTAR.,---,---,---,---,---
4,Decreto-Lei nº 9.910 de 17 de setembro de 1946,http://legislacao.planalto.gov.br/legisla/legi...,", Presidência da RepúblicaCasa CivilSubchef...",17 de Setembro de 1946,,Não consta revogação expressa,EURICO GASPAR DUTRA,Executivo,17 de Setembro de 1946,DOFC 17 09 1946 000000 0,(MRE); CREDITO SUPLEMENTAR.,---,---,---,---,---
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3733,Decreto-Lei nº 57 de 18 de novembro de 1966,http://legislacao.planalto.gov.br/legisla/legi...,", Presidência da RepúblicaCasa CivilSubchef...",18 de Novembro de 1966,ALTERA DISPOSITIVOS SOBRE LANÇAMENTO E COBRANÇ...,Não consta revogação expressa,CASTELLO BRANCO,Executivo,21 de Novembro de 1966,"DOFC DE 21/11/1966, P. 13418",MINISTÉRIO DA JUSTIÇA E NEGÓCIOS INTERIORES; M...,"a href=""http://legislacao.planalto.gov.br/legi...",---,---,---,"a href=""http://legislacao.planalto.gov.br/legi..."
3734,Decreto-Lei nº 56 de 18 de novembro de 1966,http://legislacao.planalto.gov.br/legisla/legi...,", Presidência da RepúblicaCasa CivilSubchef...",18 de Novembro de 1966,DISPÕE SOBRE A ARRECADAÇÃO DE TAXAS PELO INSTI...,Não consta revogação expressa,CASTELLO BRANCO,Executivo,18 de Novembro de 1966,DOFC 21 11 1966 013418 1,MICT. FAZENDA,,---,---,---,DEL-001712/1979 DOFC 16/11/1979 017042 2
3735,Decreto-Lei nº 55 de 18 de novembro de 1966,http://legislacao.planalto.gov.br/legisla/legi...,", Presidência da RepúblicaCasa CivilSubchef...",18 de Novembro de 1966,"DEFINE A POLÍTICA NACIONAL DE TURISMO, CRIA O ...",Revogado,CASTELLO BRANCO,Executivo,21 de Novembro de 1966,"DOFC DE 21/11/1966, P. 13416",MINISTÉRIO DAS RELAÇÕES EXTERIORES; MINISTÉRIO...,"a href=""http://legislacao.planalto.gov.br/legi...",---,---,---,"a href=""http://legislacao.planalto.gov.br/legi..."
3736,Decreto-Lei nº 54 de 18 de novembro de 1966,http://legislacao.planalto.gov.br/legisla/legi...,", Presidência da RepúblicaCasa CivilSubchef...",18 de Novembro de 1966,DA NOVA REDAÇÃO AO ART. 4º DO DECRETO-LEI 7.38...,Não consta revogação expressa,CASTELLO BRANCO,Executivo,18 de Novembro de 1966,DOFC 21 11 1966 013416 2,TELECOMUNICACAO.,,---,---,---,
